<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;"><b>Alumnes:</b> David Navarro Brugal, Antoni Llussà Sala</p>
<p style="margin: 0; text-align:right;"><b>Consultor:</b> Sergio Trilles Oliver</p>
<p style="margin: 0; text-align:right;">M2.951 · Tipologia i cicle de vida de les dades · PRA1</p>
<p style="margin: 0; text-align:right;">2019-2 · Màster universitari en Ciència de dades (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudis de Informàtica, Multimèdia i Telecomunicació</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>


# PRÀCTICA 1: Web scraping

## Codi per generar el dataset:

### Imports:

In [41]:
import requests
from urllib.parse import urlparse
import urllib
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import json
import datetime
import time
import csv
import whois
import pathlib

### Classe Throttle:

In [42]:
#llibre Web Scraping with Python
class Throttle: 
    """Add a delay between downloads to the same domain 
    """ 
    def __init__(self, delay): 
        # amount of delay between downloads for each domain 
        self.delay = delay 
        # timestamp of when a domain was last accessed 
        self.domains = {} 

    def wait(self, url): 
        domain = urlparse(url).netloc 
        last_accessed = self.domains.get(domain) 

        if self.delay > 0 and last_accessed is not None: 
            sleep_secs = self.delay - (time.time() - last_accessed) 
            if sleep_secs > 0: 
                # domain has been accessed recently 
                # so need to sleep 
                time.sleep(sleep_secs) 
        # update the last accessed time 
        self.domains[domain] = time.time() 

### Definició de funcions:

In [43]:
#Funció per fer les descàrregues dels enllaços, amb user agent i amb un control d'errors
#llibre Web Scraping with Python
def download(url, user_agent='wswp', num_retries=2): 
    print('Downloading:', url) 
    request = urllib.request.Request(url) 
    request.add_header('User-agent', user_agent)
    try: 
        html = urllib.request.urlopen(request).read() 
    except (URLError, HTTPError, ContentTooShortError) as e:
        print('Download error:', e.reason)
        html = None 
        if num_retries > 0: 
            if hasattr(e, 'code') and 500 <= e.code < 600: 
                # recursively retry 5xx HTTP errors 
                return download(url, num_retries - 1) 
    return html

#Funció per fer la descarrega dels robots.txt
#retorna el contingut dels robots.txt
def downloadRobots(base_url):
    throttle = Throttle(delay) 
    throttle.wait(base_url) 
    return download(base_url+"/robots.txt")

#Funció per fer la descarrega d'una url / pàgina
#retorna el contingut de la pàgina
def downloadPage(page):
    throttle = Throttle(delay) 
    throttle.wait(page) 
    return download(page)   

#Funció per obtenir els enllaços de totes les pàgines on hi ha els videojocs
#retorna un llistat d'enllaços
def getPages(html, base_url):
    div = html.find('div',{"id":"pagination"})
    links = div.findAll('a')
    linkPages = []
    for link in links :
        linkPages.append(base_url+link.get("href"))

    return linkPages

#Funció per obtenir els enllaços de les pàgines de detall dels videojocs
#retorna un llistat d'enllaços de les pàgines del detall amb la url completa segons el base_url
def getLinkGames(html,base_url):
    divs = html.findAll('div',{"class":"list-item-title"})
    linkGames = []
    for div in divs :
        linkGames.append(base_url+div.a.get("href"))
    return np.unique(linkGames)

#Funció per obtenir el soup de la url del videojoc
def getHtmlGame(urlGame):
    reqGame = requests.get(urlGame)
    htmlGame = BeautifulSoup(reqGame.text, "html5lib")
    return htmlGame

#Funció per obtenir el contingut d'strongs de la classe quote
def getQuoteStrongs(soup):
    p = soup.find('p',{"class":"quote"})
    strongsOut = []
    dict = {}
    if(p):
        strongs = p.findAll('strong')
        for strong in strongs :
            title = strong.text
            html= ""
            if "Rating" in title:
                for tag in strong.next_siblings:
                    if tag.name == "strong":
                        break
                    else:
                        html = str(tag).replace("<br/>","")
                        listRating = html.split(':')
                        if(len(listRating) == 2):
                            dict["Ratings-"+listRating[0].strip()] = listRating[1].strip()
            else:
                html = strong.next_sibling.next_sibling
                if(title.count("Total") > 0):
                    dict["Ratings-Total"] = str(title).replace("Total","").replace(":","").strip()
                elif(title.count("Overall") > 0):
                    dict["Ratings-Total"] = str(title).replace("Overall:","").replace(":","").strip()
                else:
                    dict[title] = str(html).strip()
    return dict

#Funció per obtenir els attributs importants: del costat de la imatge principal
def getImportantAttributes(soup):
    div = soup.find('div',{"id":"important"})
    dict = {}
    h1 = div.find('h1')
    dict["title"] = h1.text
    h3List = div.findAll('h3')
    h4List = div.findAll('h4')
    idDate = 0;
    for h3 in h3List :
        name =h3.get("class")[0]
        if h3.text:
            dict[name]=h3.text.replace('\t','').replace('\n','')
    for h4 in h4List :
        name =h4.get("class")[0]
        if h4.text:
            if(name == "date"): #com que el genre també te la classe date, s'ha de controlar per etiquetar-ho correctament
                idDate +=1;
            if(idDate > 1):
                name= "genre"
            dict[name] = h4.text.replace('\t','').replace('\n','')
    return dict
            

#Funció per obtenir la descripció i referència
def getDescriptionAndReferences(soup):
    pList = soup.findAll('p',{"class":["description", "references"]})
    dict = {}
    for p in pList :
        name = p.get("class")[0]
        if (p.text != ''):
            dict[name] = p.text.replace('\t','').replace('\n','')
    return dict
    

#Funció per obtenir els enllaços de Youtube i els seus títols.
def getYoutubeVideos(soup):
    divList = soup.findAll('div',{"class":"media"})
    dict = {}
    listLink = []
    listTitle = []
    for div in divList :
        link = div.find('iframe')["src"]
        listLink.append(link)
        dict["youtube-links"] = ", ".join(map(str, listLink))
        title = div.h5.text
        listTitle.append(title)
        dict["youtube-titles"] = ", ".join(map(str, listTitle))
    return dict


#Funció per obtenir els enllaços de Game-screenshots i els seus títols.
def getGameScreenShots(soup):
    divList = soup.findAll('div',{"class":"top-game single-game"})
    dict = {}
    listLink = []
    listTitle = []
    for div in divList :
        link = div.find ('img')["src"] 
        listLink.append(link)
        dict["screenshots-links"] = ", ".join(map(str, listLink))
        title = div.find('div',{"class": "top-game-text"})
        title1 = title.p.text
        listTitle.append(title1)
        dict["screenshots-titles"] = ", ".join(map(str, listTitle))
        load_requests(link, title1)
    return dict


#Funció que emmagatzema la imatge amb url a una carpeta.    
def load_requests(source_url, title1):
    r = requests.get(source_url, stream = True)
    if r.status_code == 200:
        aSplit = source_url.split('/')
        ruta = rutaIMGS1+title1+"/"+aSplit[len(aSplit)-1] 
        pathlib.Path(rutaIMGS1+title1).mkdir(parents=True, exist_ok=True)
        #print(ruta)
        output = open(ruta,"wb")
    for chunk in r:
        output.write(chunk)
    output.close()
    
#Funció per obtenir la informació de les roms
def getRomsInfo(soup):
    pList = soup.findAll('p',{"class":"rom-info"})
    dict = {}
    for p in pList :
        strongList = p.findAll('strong',{"class":"info-name"})
        country = ""
        for strong in strongList:
            title = strong.text
            if title == "Country":
                country = strong.next_sibling.next_sibling.text.replace('\xa0','')
            if title == "Common Filenames":
                listCommonFiles = []
                for tag in strong.next_siblings:
                    if(tag):
                        commonFiles = str(tag).strip().replace("<em>","").replace("</em>","").replace("<br/>","")
                        if(commonFiles):
                            listCommonFiles.append(commonFiles)
                desc = ", ".join(map(str, listCommonFiles))
            else:    
                desc = strong.next_sibling.next_sibling.text.replace('\xa0','')
            dict[title+"-"+country]= desc
    return dict

#Funció per obtenir la informació de la gent d'staff
def getStaffInfo(soup):
    pList = soup.findAll('p',{"class":"staff-list"})
    dict = {}
    listPeople=[]
    for p in pList :
        strongList = p.findAll('strong')
        for strong in strongList:
            title = strong.text
            listPeople = []
            for tag in strong.next_siblings:
                if tag.name == "strong":
                    break
                else:
                    people = str(tag).strip().replace("<em>","").replace("</em>","").replace("<br/>","")
                    if(people):
                        listPeople.append(people)
            dict[title] = ", ".join(map(str, listPeople))
    return dict    
    

### Definició de constants:

In [44]:
#Constants

base_url = "https://superfamicom.org"
path_snes = "/game-list"
delay=0.5
#ruta segons usuari
rutaIMGS="E:/UOC/Tipologia i cicle de vida de les dades/UOC-PRA1-M2.951-WebScraping-dnavarro30-tllusa/imgs/"
rutaIMGS1 = "/Users/davidnavarrobrugal/Documents/imatges_jocs/"



### Propietàri de la pàgina:

In [45]:
print (whois.whois("https://superfamicom.org"))

{
  "domain_name": [
    "SUPERFAMICOM.ORG",
    "superfamicom.org"
  ],
  "registrar": "NAMECHEAP INC",
  "whois_server": "whois.namecheap.com",
  "referral_url": null,
  "updated_date": [
    "2018-09-26 15:23:23",
    "2015-12-14 23:55:32"
  ],
  "creation_date": "2008-07-17 14:09:41",
  "expiration_date": "2024-07-17 14:09:41",
  "name_servers": [
    "NS1.DREAMHOST.COM",
    "NS2.DREAMHOST.COM",
    "NS3.DREAMHOST.COM",
    "ns1.dreamhost.com",
    "ns2.dreamhost.com",
    "ns3.dreamhost.com"
  ],
  "status": "ok https://icann.org/epp#ok",
  "emails": [
    "abuse@namecheap.com",
    "gapetto@gmail.com"
  ],
  "dnssec": "unsigned",
  "name": "Matthew  Callis",
  "org": "eludevisibility",
  "address": "625 Bel Air Dr.",
  "city": "Nashville",
  "state": "S",
  "zipcode": "37217",
  "country": "US"
}


### Comprovació dels robots:

In [46]:
robots = downloadRobots(base_url)
print(robots)

Downloading: https://superfamicom.org/robots.txt
b'Sitemap: http://superfamicom.org/sitemap.xml\nSitemap: http://superfamicom.org/sitemap-famicom.xml\nSitemap: http://superfamicom.org/sitemap-translations.xml\nSitemap: http://superfamicom.org/blog/sitemap.xml\nUser-agent: *\nDisallow: /cgi-bin/\nDisallow: /hop/\nDisallow: /cache/\nDisallow: *function.*\nDisallow: /sitemap.php'


No s'han trobat restriccions per poder realitzar el webscraping.

### Aconseguir els enllaços de les diferents pàgines:

In [47]:
html = downloadPage(base_url+path_snes)
soup = BeautifulSoup(html, "html5lib")
linkPages = getPages(soup,base_url)

print(linkPages)

Downloading: https://superfamicom.org/game-list
['https://superfamicom.org/game-list', 'https://superfamicom.org/game-list/2', 'https://superfamicom.org/game-list/3', 'https://superfamicom.org/game-list/4', 'https://superfamicom.org/game-list/5', 'https://superfamicom.org/game-list/6', 'https://superfamicom.org/game-list/7', 'https://superfamicom.org/game-list/8', 'https://superfamicom.org/game-list/9', 'https://superfamicom.org/game-list/10', 'https://superfamicom.org/game-list/11', 'https://superfamicom.org/game-list/12', 'https://superfamicom.org/game-list/13', 'https://superfamicom.org/game-list/14', 'https://superfamicom.org/game-list/15', 'https://superfamicom.org/game-list/16', 'https://superfamicom.org/game-list/17', 'https://superfamicom.org/game-list/18', 'https://superfamicom.org/game-list/19', 'https://superfamicom.org/game-list/20', 'https://superfamicom.org/game-list/21', 'https://superfamicom.org/game-list/22', 'https://superfamicom.org/game-list/23', 'https://superfamic

### Aconseguir els enllaços de tots els videojocs, de totes les pàgines:

In [48]:
linkGames = []
for linkPage in linkPages:
    html = downloadPage(linkPage)
    soup = BeautifulSoup(html, "html5lib")
    incomingLinkGames = getLinkGames(soup,base_url)
    for link in incomingLinkGames :
        linkGames.append(link)
print(linkGames)  
    

Downloading: https://superfamicom.org/game-list
Downloading: https://superfamicom.org/game-list/2
Downloading: https://superfamicom.org/game-list/3
Downloading: https://superfamicom.org/game-list/4
Downloading: https://superfamicom.org/game-list/5
Downloading: https://superfamicom.org/game-list/6
Downloading: https://superfamicom.org/game-list/7
Downloading: https://superfamicom.org/game-list/8
Downloading: https://superfamicom.org/game-list/9
Downloading: https://superfamicom.org/game-list/10
Downloading: https://superfamicom.org/game-list/11
Downloading: https://superfamicom.org/game-list/12
Downloading: https://superfamicom.org/game-list/13
Downloading: https://superfamicom.org/game-list/14
Downloading: https://superfamicom.org/game-list/15
Downloading: https://superfamicom.org/game-list/16
Downloading: https://superfamicom.org/game-list/17
Downloading: https://superfamicom.org/game-list/18
Downloading: https://superfamicom.org/game-list/19
Downloading: https://superfamicom.org/game

### Procés de descarrega del contingut de les pàgina del videojoc:

In [49]:
from time import time
start = time()
games = []

listQuotes = []
listImportantAttributes = []
listDescriptionAndReferences = []
listScreenShots = []
listYoutube = []
listRomsInfo = []
listStaffInfo = []

countGames = 0
for linkGame in linkGames:
    soupGame = getHtmlGame(linkGame)
    
    quotes = getQuoteStrongs(soupGame)
    importantAttributes = getImportantAttributes(soupGame)
    youtube = getYoutubeVideos(soupGame)
    descriptionAndReferences = getDescriptionAndReferences(soupGame)
    screenShots = getGameScreenShots(soupGame)
    romsInfo = getRomsInfo(soupGame)
    staffInfo = getStaffInfo(soupGame)

    joinInfo = {**importantAttributes , **quotes , **romsInfo, **descriptionAndReferences, ** staffInfo, **youtube, **screenShots }
#     joinInfo = importantAttributes + descriptionAndReferences + quotes + youtube + screenShots+ romsInfo + staffInfo
    
    games.append(joinInfo)
    listQuotes.append(quotes)
    listImportantAttributes.append(importantAttributes)
    listDescriptionAndReferences.append(descriptionAndReferences)
    listScreenShots.append(screenShots)
    listYoutube.append(youtube)
    listRomsInfo.append(romsInfo)
    listStaffInfo.append(staffInfo)
    
    countGames += 1

end = time()
diff_time1 = end - start
print("Temps d'execució de la descarrega dels jocs {} segons".format(diff_time1))
print ("Total jocs: ",countGames)

    
    

Temps d'execució de la descarrega dels jocs 4992.908461093903 segons
Total jocs:  1868


### Creació d'un dataframe

In [50]:
df = pd.DataFrame()
for game in games:
    df = df.append(game, ignore_index=True)

### Preparació per la sortida cap a .csv

In [51]:
df.head()

df.to_csv('superfamicon.csv',sep=";", encoding="utf-8",index=False)

### Verificació de la informació

In [52]:
dfImportantAttributes = pd.DataFrame()
for obj in listImportantAttributes:
    dfImportantAttributes = dfImportantAttributes.append(obj, ignore_index=True)
    

dfImportantAttributes.to_csv('dfImportantAttributes.csv',sep=";", encoding="utf-8",index=False)

In [53]:
dfDescriptionAndReferences = pd.DataFrame()
for obj in listDescriptionAndReferences:
    dfDescriptionAndReferences = dfDescriptionAndReferences.append(obj, ignore_index=True)

dfDescriptionAndReferences.to_csv('dfDescriptionAndReferences.csv',sep=";", encoding="utf-8",index=False)

In [54]:
dfRomsInfo = pd.DataFrame()
for obj in listRomsInfo:
    dfRomsInfo = dfRomsInfo.append(obj, ignore_index=True)

dfRomsInfo.to_csv('dfRomsInfo.csv',sep=";", encoding="utf-8",index=False)

In [55]:
dfStaffInfo = pd.DataFrame()
for obj in listStaffInfo:
    dfStaffInfo = dfStaffInfo.append(obj, ignore_index=True)

dfStaffInfo.to_csv('dfStaffInfo.csv',sep=";", encoding="utf-8",index=False)

In [56]:
dfQuotes = pd.DataFrame()
for obj in listQuotes:
    dfQuotes = dfQuotes.append(obj, ignore_index=True)

dfQuotes.to_csv('dfQuotes.csv',sep=";", encoding="utf-8",index=False)

In [57]:
dfScreenShots = pd.DataFrame()
for obj in listScreenShots:
    dfScreenShots = dfScreenShots.append(obj, ignore_index=True)

dfScreenShots.to_csv('dfScreenShots.csv',sep=";", encoding="utf-8",index=False)

In [58]:
dfYoutube = pd.DataFrame()
for obj in listYoutube:
    dfYoutube = dfYoutube.append(obj, ignore_index=True)

dfYoutube.to_csv('dfYoutube.csv',sep=";", encoding="utf-8",index=False)